In [13]:
import boto3
import pandas as pd
import json
from urllib.request import urlopen

In [14]:
with open('cred') as f:
    lines = f.readlines()
key_id = (lines[0])[:-1]
secret_key = lines[1]

In [15]:
url = "https://bank.gov.ua/NBU_Exchange/exchange_site?start=20210101&end=20211231&valcode=usd&sort=exchangedate&order=desc&json"
response = urlopen(url)

In [16]:
url2 = "https://bank.gov.ua/NBU_Exchange/exchange_site?start=20210101&end=20211231&valcode=eur&sort=exchangedate&order=desc&json"
response_eur = urlopen(url2)

In [17]:
df2= pd.DataFrame.from_records(json.loads(response_eur.read()))

In [18]:
data_json = json.loads(response.read())

In [19]:
df = pd.DataFrame.from_records(data_json)

In [20]:
df = df.append(df2)

In [21]:
def to_date(row):
    date_string = row["exchangedate"]
    return pd.to_datetime(date_string,
               format="%d.%m.%Y")

In [22]:
df['date'] = df.apply(to_date, axis=1)

In [23]:
df = df.sort_values(by='date')

In [24]:
df.to_csv("cur_df.csv",index=False)

In [25]:

s3 = boto3.resource('s3', aws_access_key_id = key_id, aws_secret_access_key
 = secret_key)
s3.meta.client.upload_file('cur_df.csv', 'truezhesha', 'cur_df.csv')

In [ ]:

s3 = boto3.resource('s3', aws_access_key_id = key_id, aws_secret_access_key
 = secret_key)
s3.meta.client.upload_file('graph.csv', 'truezhesha', 'cur_df.csv')